## Overview

This notebook was created by 
* Heejung Jung
* Xiaochun Han
* Deepanshi 

Project: MVPC – classify behaving animal data in searchlights using SVM
* P1. 1 out of 20 conditions
* P2. Classify taxa (1 out of 5), training on videos with 3 behaviors and testing on videos with the left-out behavior.
* P3. Classify behaviors (1 out of 4), training on videos with 4 taxa and testing on videos with left-out taxonomic category

Parameters:
* The attention task (tax, beh) was ignored and was treated as separate runs. There were 5 runs per task, however, we've treated them as 10 runs. 
* search light radii: 10

In [1]:
# load libraries
import os
import numpy as np
import mvpa2
import pandas as pd
# from mvpa2.tutorial_suite import *
import nilearn
import nipy
import mvpa2.suite as mv
import nibabel
from numpy.testing.decorators import skipif


/Users/h/anaconda3/envs/pymvpa_eng/lib/python2.7/site-packages/pydicom/__init__.py:55: DeprecationWarning: Python 2 will no longer be supported after the pydicom v1.4 release
  warnings.warn(msg, DeprecationWarning)
/Users/h/anaconda3/envs/pymvpa_eng/lib/python2.7/site-packages/nilearn/__init__.py:73: DeprecationWarning: Python2 support is deprecated and will be removed in the next release. Consider switching to Python 3.6 or 3.7.
  _python_deprecation_warnings()
/Users/h/anaconda3/envs/pymvpa_eng/lib/python2.7/site-packages/_pytest/mark/structures.py:383: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)
/Users/h/anaconda3/envs/pymvpa_eng/lib/python2.7/site-packages/nipy/testing/decorators.py:11: DeprecationWarning: Importing from numpy.testing.decorators is deprecated since numpy 1.15.0, import from numpy.testing instead.
  from numpy.testing.decorators import *
/User

### helpful resources

http://www.pymvpa.org/examples/searchlight_surf.html


class mvpa2.datasets.gifti.AttrDataset(samples, sa=None, fa=None, a=None)

A dataset consists of four pieces. The core is a two-dimensional array that has variables (so-called features) in its columns and the associated observations (so-called samples) in the rows. In addition a dataset may have any number of attributes for features and samples. Unsurprisingly, these are called ‘feature attributes’ and ‘sample attributes’. Each attribute is a vector of any datatype that contains a value per each item (feature or sample). Both types of attributes are organized in their respective collections – accessible via the sa (sample attribute) and fa (feature attribute) attributes. Finally, a dataset itself may have any number of additional attributes (i.e. a mapper) that are stored in their own collection that is accessible via the a attribute (see examples below).

# P1. 1 out of 20

### load dataset (glm output)

In [3]:
main_dir = '/Users/h/Documents/projects_local/cluster_projects'
# main_dir = '/dartfs-hpc/scratch/psyc164/groupXHD'
sub_name = 'sub-rid000001'
task_list = ['beh', 'tax']
data_set = []
for task_name in task_list:
    for run_num in range(1,6):
        ds=[]
#         tax_beh = ["bird_eating", "bird_fighting", "bird_running", "bird_swimming",
#                            "insect_eating", "insect_fighting", "insect_running", "insect_swimming",
#                            "primate_eating","primate_fighting","primate_running","primate_swimming",
#                            "reptile_eating","reptile_fighting", "reptile_running", "reptile_swimming",
#                            "ungulate_eating","ungulate_fighting","ungulate_running","ungulate_swimming"]
#         beh = np.tile(['eating','fighting','running','swimming'],5)
#         tax = np.repeat(['bird','insect','primate', 'reptile', 'ungulate'],4)
#         run = np.repeat(run_num, 20)
        
        gifti_fname = os.path.join(main_dir,'analysis', sub_name,'func', sub_name + '_task-' + task_name + '_run-' + str(run_num) +'_rw-glm.lh.coefs.gii')
        ds = mv.gifti_dataset(gifti_fname)
        
        # order in sub-rid000001_task-beh_run-5_rw-glm.lh.X.xmat.1D 
        ds.sa['beh_tax'] = ["bird_eating", "bird_fighting", "bird_running", "bird_swimming",
                           "insect_eating", "insect_fighting", "insect_running", "insect_swimming",
                           "primate_eating","primate_fighting","primate_running","primate_swimming",
                           "reptile_eating","reptile_fighting", "reptile_running", "reptile_swimming",
                           "ungulate_eating","ungulate_fighting","ungulate_running","ungulate_swimming"]
        ds.sa['beh'] = np.tile(['eating','fighting','running','swimming'],5)
        ds.sa['tax'] = np.repeat(['bird','insect','primate', 'reptile', 'ungulate'],4)
        ds.sa['run'] = np.repeat(run_num, ds.shape[0])
        ds.fa['node_indices'] = range(0,ds.shape[1]) # 0 ~ 400000
        data_set.append(ds) 
    
# http://www.pymvpa.org/tutorial_mappers.html
within_ds = mv.vstack(data_set)

/Users/h/anaconda3/envs/pymvpa_eng/lib/python2.7/site-packages/mvpa2/datasets/gifti.py:77: DeprecationWarning: giftiio.read function deprecated. Use nibabel.load() instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  samples = giftiio.read(samples)


In [4]:
within_ds.shape

(200, 40962)

In [5]:
within_ds.fa

FeatureAttributesCollection(items=[ArrayCollectable(name='node_indices', doc=None, value=array([    0,     1,     2, ..., 40959, 40960, 40961]), length=40962)])

### ds parameters (re-number the run/chunks)

In [6]:
ds_q1 = within_ds.copy()

In [7]:
ds_q1.sa['chunks'] = np.repeat(range(1,11), ds.shape[0]) 
ds_q1.sa['targets'] = ds_q1.sa['beh_tax']
del ds_q1.sa['intents']

In [16]:
mv.zscore(ds_q1)

In [17]:
ds_q1.fa.node_indices

array([    0,     1,     2, ..., 40959, 40960, 40961])

## classification

In [18]:
# setting up classifier
clf = mv.LinearCSVMC() #space = 'targets'
# cv = mv.CrossValidation(clf, mv.NFoldPartitioner())
cv = mv.CrossValidation(clf, mv.NFoldPartitioner()) #attr='chunks', , errorfx=mv.mean_match_accuracy

In [19]:
# cross validation (run as chunks) on one participants' glm gifti
cv_within = cv(ds_q1)

# why is the mean lower?

/Users/h/anaconda3/envs/pymvpa_eng/lib/python2.7/site-packages/mvpa2/misc/attrmap.py:153: FutureWarning: Conversion of the second argument of issubdtype from `str` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype(str).type`.
  if not np.issubdtype(attr.dtype, str) and not self.mapnumeric:
/Users/h/anaconda3/envs/pymvpa_eng/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:215: FutureWarning: Conversion of the second argument of issubdtype from `'c'` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype('c').type`.
  if ( np.issubdtype(self.ca.trained_targets.dtype, 'c') or


In [20]:
cv_within

Dataset(array([[0.5 ],
       [0.45],
       [0.3 ],
       [0.35],
       [0.45],
       [0.4 ],
       [0.65],
       [0.5 ],
       [0.6 ],
       [0.6 ]]), sa=SampleAttributesCollection(items=[ArrayCollectable(name='cvfolds', doc=None, value=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), length=10)]), fa=FeatureAttributesCollection(items=[]), a=DatasetAttributesCollection(items=[]))

In [21]:
np.mean(cv_within)

0.47999999999999987

## searchlight 
* reference: http://www.pymvpa.org/PyMVPA-Manual.pdf

* converting lh.pial into lh.pial.gii <br>
```/opt/freesurfer/5.3.0/subjects/fsaverage6/surf``` <br>
```mris_convert lh.pial /dartfs-hpc/scratch/psyc164/groupXHD/lh.pial.gii```

In [22]:
# version 1
fsaverage_gii = '/Users/h/Documents/projects_local/cluster_projects/fs_templates/lh.pial.gii'
# fsaverage_gii = '/Users/h/Documents/projects_local/cluster_projects/sub-rid000001_T1w_pial.L.surf.gii'
# surf = nibabel.gifti.read(fsaverage_gii)
surf = mv.surf.read(fsaverage_gii)
# note: surf.vertices.shape (81920, 3) and surf.faces.shape (40962, 3) surface = surf, 
qe = mv.SurfaceQueryEngine(surf, radius = 10.0, 
                           distance_metric='dijkstra')

# enable_ca=['roi_sizes'],roi_ids=cortical_vertices

sl = mv.Searchlight(cv, queryengine=qe, nproc=1)

/Users/h/anaconda3/envs/pymvpa_eng/lib/python2.7/site-packages/mvpa2/support/nibabel/surf_gifti.py:51: DeprecationWarning: giftiio.read function deprecated. Use nibabel.load() instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  g = giftiio.read(fn)
/Users/h/anaconda3/envs/pymvpa_eng/lib/python2.7/site-packages/mvpa2/support/nibabel/surf_gifti.py:25: DeprecationWarning: getArraysFromIntent method deprecated. Use get_arrays_from_intent instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  ar = g.getArraysFromIntent(intent)


In [26]:
sl_q1 = sl(ds_q1)

 * Please note: warnings are printed only once, but underlying problem might occur many times *


KeyboardInterrupt: 

In [25]:
searchlight_q1_filename = os.path.join(main_dir, 'analysis', 'searchlight', sub_name + '.gii')
nimg = mv.map2gifti(fwm, filename, encoding='GIFTI_ENCODING_B64GZ', surface=surf)

NameError: name 'fwm' is not defined

# Classify taxa (1 out of 5), training on videos with 3 behaviors and testing on videos with the left-out behavior.
* chunk is now behavior
* target is taxa

In [397]:
# classify on taxanomy, cv with behavior
ds_q2 = within_ds.copy()
ds_q2.sa['chunks'] = ds_q2.sa['beh']
ds_q2.sa['target'] = ds_q2.sa['tax']
del ds_q2.sa['intents']

In [404]:
ds_q2

Dataset(array([[  3.9534876 ,  -0.43695655, -10.585636  , ...,   4.8139644 ,
          0.20275442,  -1.1942437 ],
       [  1.2697178 ,  -7.199501  , -11.044861  , ...,  -3.4059372 ,
         -1.4120976 ,  -1.1120566 ],
       [  4.230085  ,  -2.5236893 , -17.407463  , ...,  -6.259083  ,
         -6.1621137 ,  -2.53753   ],
       ...,
       [  2.4272716 ,  -4.728557  , -20.483679  , ...,  -9.10487   ,
         -8.90564   ,  -2.8355129 ],
       [ -6.7157383 ,  -5.1460204 ,  -2.2242303 , ...,   8.366821  ,
          3.2633743 ,  -6.341244  ],
       [ -0.04155796,  -3.311609  ,   0.74648255, ...,  10.623682  ,
          7.109033  ,   2.0199761 ]], dtype=float32), sa=SampleAttributesCollection(items=[ArrayCollectable(name='run', doc=None, value=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4

In [402]:
# classifier
clf = mv.LinearCSVMC()
cv = mv.CrossValidation(clf, mv.NFoldPartitioner(), enable_ca=['stats'])

In [403]:
cv_within_q2 = cv(ds_q2)
cv_within_q2

Dataset(array([[0.82],
       [0.8 ],
       [0.8 ],
       [0.74]]), sa=SampleAttributesCollection(items=[ArrayCollectable(name='cvfolds', doc=None, value=array([0, 1, 2, 3]), length=4)]), fa=FeatureAttributesCollection(items=[]), a=DatasetAttributesCollection(items=[]))

In [411]:
np.mean(cv_within_q2)

0.79

In [ ]:
# searchlight
roi_ids = None
sl = Searchlight(cv, queryengine=qe , postproc=mean_sample(), roi_ids=roi_ids)
sl_ds_q2 = sl(ds_q2)

# Classify behaviors (1 out of 4), training on videos with 4 taxa and testing on videos with left-out taxonomic category

In [409]:
# classify on behavior, cv with taxonomy
ds_q3 = within_ds.copy()
ds_q3.sa['chunks'] = ds_q3.sa['tax']
ds_q3.sa['targets'] = ds_q3.sa['beh']
del ds_q3.sa['intents']

In [410]:
# classifier
clf = mv.LinearCSVMC()
cv = mv.CrossValidation(clf, mv.NFoldPartitioner(), enable_ca=['stats'])
cv_within_q3 = cv(ds_q3)
cv_within_q3

Dataset(array([[0.275],
       [0.325],
       [0.225],
       [0.2  ],
       [0.35 ]]), sa=SampleAttributesCollection(items=[ArrayCollectable(name='cvfolds', doc=None, value=array([0, 1, 2, 3, 4]), length=5)]), fa=FeatureAttributesCollection(items=[]), a=DatasetAttributesCollection(items=[]))

In [412]:
np.mean(cv_within_q3)

0.275

In [415]:
roi_ids = None
sl = mv.Searchlight(cv, queryengine=qe , postproc=mv.mean_sample(), roi_ids=roi_ids)
sl_ds_q3 = sl(ds_q3)

KeyboardInterrupt: 

# directly from search_clf

In [29]:
import sys
from os import mkdir
from os.path import exists, join
import numpy as np
import mvpa2.suite as mv
runs = [1, 2, 3, 4, 5]
n_conditions = 20
n_vertices = 40962
#n_medial = {'lh': 3487, 'rh': 3491}
n_medial = {'lh': 3486, 'rh': 3491}

# Load surface and create searchlight query engine
fsaverage_gii = '/Users/h/Documents/projects_local/cluster_projects/fs_templates/lh.pial.gii'
# fsaverage_gii = '/Users/h/Documents/projects_local/cluster_projects/sub-rid000001_T1w_pial.L.surf.gii'
# surf = nibabel.gifti.read(fsaverage_gii)
surf = mv.surf.read(fsaverage_gii)

# surf = mv.surf.read(join(suma_dir, '{0}.pial.gii'.format(hemi)))
qe = mv.SurfaceQueryEngine(surf, 10.0, distance_metric='dijkstra')
glm_dir = '/Users/h/Documents/projects_local/cluster_projects/analysis/'
participant = 'rid000001'
task = 'beh'
hemi = 'lh'
# Load in surface data sets
dss = []

for run in runs:
    ds = mv.niml.read(join(glm_dir, 'sub-{0}'.format(participant), 'func',
                       'sub-{0}_task-{1}_run-{2}_rw-glm.{3}.coefs.niml.dset'.format(
                            participant, task, run, hemi)))
    ds.sa.pop('stats')
    ds.sa['behavior'] = np.tile(['eating', 'fighting', 'running', 'swimming'], 5)
    ds.sa['taxonomy'] = np.repeat(['bird', 'insect', 'primate', 'reptile', 'ungulate'], 4)
    ds.sa['conditions'] = [' '.join((tax, beh)) for tax, beh in zip(ds.sa.taxonomy, ds.sa.behavior)]
    for lab, cond in zip(ds.sa.labels, ds.sa.conditions):
        assert ' '.join(lab.split('#')[0].split('_')) == cond
    ds.sa['runs'] = [run] * 20
    ds.sa['subjects'] = [participant] * 20
    ds.fa['node_indices'] = range(n_vertices)
    dss.append(ds)
ds = mv.vstack(dss)



In [41]:
# # Exclude medial wall
# medial_wall = np.where(np.sum(ds.samples == 0, axis=0) == n_conditions * 5)[0].tolist()
# cortical_vertices = np.where(np.sum(ds.samples == 0, axis=0) < n_conditions * 5)[0].tolist()
# assert len(medial_wall) == n_medial[hemi]
# assert len(medial_wall) + len(cortical_vertices) == n_vertices

#np.save(join(mvpa_dir, 'cortical_vertices_{0}.npy'.format(hemi)), cortical_vertices)
#cortical_vertices = = np.load(join(mvpa_dir, 'cortical_vertices_{0}.npy').tolist()

# Z-score features across samples
#mv.zscore(ds, chunks_attr='runs')
ds.samples = ((ds.samples - np.mean(ds.samples, axis=1)[:, None])
              / np.std(ds.samples, axis=1)[:, None])

clf = mv.LinearCSVMC(space='conditions')

cv = mv.CrossValidation(clf, mv.NFoldPartitioner(attr='runs'), errorfx=mv.mean_match_accuracy)



In [42]:
ds.sa

SampleAttributesCollection(items=[ArrayCollectable(name='runs', doc=None, value=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]), length=100), ArrayCollectable(name='taxonomy', doc=None, value=array(['bird', 'bird', 'bird', 'bird', 'insect', 'insect', 'insect',
       'insect', 'primate', 'primate', 'primate', 'primate', 'reptile',
       'reptile', 'reptile', 'reptile', 'ungulate', 'ungulate',
       'ungulate', 'ungulate', 'bird', 'bird', 'bird', 'bird', 'insect',
       'insect', 'insect', 'insect', 'primate', 'primate', 'primate',
       'primate', 'reptile', 'reptile', 'reptile', 'reptile', 'ungulate',
       'ungulate', 'ungulate', 'ungulate', 'bird', 'bird', 'bird', 'bird',
       'insect', 'insec

In [43]:
sl = mv.Searchlight(cv, queryengine=qe, enable_ca=['roi_sizes'],
                    nproc=1, roi_ids=cortical_vertices)

In [ ]:
sl_result = sl(ds)


In [492]:
len(sl.roi_ids)

37476

In [ ]:
searchlight_q1_filename = os.path.join(main_dir, 'analysis', 'searchlight', sub_name + '.gii')
nimg = mv.map2gifti(fwm, filename, encoding='GIFTI_ENCODING_B64GZ', surface=surf)